In [ ]:
#The first step is to import all the necessary libraries

import pandas as pd
import geopandas as gpd
import folium
import osmnx as ox
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
# Then loading in the Gowalla Cambridge dataset as a Pandas dataframe
df = pd.read_csv('Cambridge_gowalla.csv')

In [ ]:
# Here, A new dataframe is being created to include only check-ins by users 75027 and 102829 as that is our target for the first question
user_df = df[df['User_ID'].isin([75027, 102829])]

In [ ]:
# Converting the dataframe to a GeoPandas dataframe to plot on an interactive folium map
# Columns lon and lat are the geometry for the dataframe
gdf = gpd.GeoDataFrame(user_df, geometry=gpd.points_from_xy(user_df.lon, user_df.lat))

In [ ]:
gdf.head()

In [ ]:
# Using the geocode function to obtain the coordinates of Cambridge 
ox.geocode('Cambridge, UK')

In [ ]:
# As mentioned earlier, a folium map is generated using the coordinates of Cambridge, with a reasonable zoom level
cambridge_coords = [52.2055314, 0.1186637]
map = folium.Map(location=cambridge_coords, zoom_start=13)

In [ ]:
# Using a loop and the folium.marker function to generate the check in locations on a map
# To make the check in points easier to diferentiate, each user's check-ins are marked with different colours
for user_id, color in zip([75027, 102829], ['red', 'blue']):
    user_gdf = gdf[gdf['User_ID'] == user_id]
    for _, row in user_gdf.iterrows():
        folium.Marker(location=[row['lat'], row['lon']], icon=folium.Icon(color=color)).add_to(map)

# Red here and blue represent the check-in points of users 75027 and 10829 respectively

# Finally, the map will be displayed
map

In [ ]:
# Creating a new dataframe to include points for user 75027 and only their movements on 30th January 2010
user_75027 = df[(df['User_ID'] == 75027) & (df['date'].str.startswith('30/01/2010'))]

# Obtaining the address of cambrige
graph = ox.graph_from_place('Cambridge, United Kingdom', network_type='all')

import matplotlib.pyplot as plt

fig, ax = ox.plot_graph(graph, show=False, close=False, edge_color='lightgray', node_color='none', figsize=(8,8))
ax.scatter(user_75027['lon'], user_75027['lat'], color='red', zorder=3, s=50)

# A loop here defines the start node and end node, where as the route here plots the path between them. This iterates for all the points in the dataframe
for i in range(len(user_75027)-1):
    start = ox.nearest_nodes(graph, user_75027.iloc[i]['lon'], user_75027.iloc[i]['lat'])
    end = ox.nearest_nodes(graph, user_75027.iloc[i+1]['lon'], user_75027.iloc[i+1]['lat'])
    route = ox.shortest_path(graph, start, end, weight='length')
    
    
    ox.plot_graph_route(graph, route, ax=ax, route_color='green', route_linewidth=5, node_size=0, show=False, close=False)
plt.show()

In [ ]:
# The same steps are repeated for user 10289

user_102829 = df[(df['User_ID'] == 102829) & (df['date'].str.startswith('24/05/2010'))]
graph = ox.graph_from_place('Cambridge, United Kingdom', network_type='all')

import matplotlib.pyplot as plt

fig, ax = ox.plot_graph(graph, show=False, close=False, edge_color='lightgray', node_color='none', figsize=(8,8))
ax.scatter(user_102829['lon'], user_102829['lat'], color='blue', zorder=3, s=50)


for i in range(len(user_102829)-1):
    start = ox.nearest_nodes(graph, user_102829.iloc[i]['lon'], user_102829.iloc[i]['lat'])
    end = ox.nearest_nodes(graph, user_102829.iloc[i+1]['lon'], user_102829.iloc[i+1]['lat'])
    route = ox.shortest_path(graph, start, end, weight='length')
    
    
    ox.plot_graph_route(graph, route, ax=ax, route_color='green', route_linewidth=5, node_size=0, show=False, close=False)
plt.show()

In [ ]:
from geopy.distance import geodesic

In [ ]:
# Creating a new list with the displacement between each consecutive pair of latitude/longitude coordinates using a loop similar to the one made earlier for the shortest path
# The geodesic distance here allows the distanfe to be measures in kilometers
displacements = []
for i in range(1, len(user_75027)):
    prev_lat, prev_lon = user_75027.iloc[i-1]['lat'], user_75027.iloc[i-1]['lon']
    curr_lat, curr_lon = user_75027.iloc[i]['lat'], user_75027.iloc[i]['lon']
    displacement = geodesic((prev_lat, prev_lon), (curr_lat, curr_lon)).km
    displacements.append(displacement)

# Using the max command on the list provides the maximum displacement value
max_displacement = max(displacements)
print(f"Maximum displacement for user 75027 on 30/01/2010 is {max_displacement:.2f} km.")

# The average displacement for the user is calculated by dividing the sum of displacements by the total points
avg_displacement = sum(displacements) / len(displacements)
print(f"Average displacement for user 75027 on 30/01/2010 is {avg_displacement:.2f} km.")

# Summing the list gives the total displacement for the user
total_displacement = sum(displacements)
print(f"Total displacement for user 75027 on 30/01/2010 is {total_displacement:.2f} km.")

In [ ]:
# The same steps are repeated for user 102829
displacements = []
for i in range(1, len(user_102829)):
    prev_lat, prev_lon = user_102829.iloc[i-1]['lat'], user_102829.iloc[i-1]['lon']
    curr_lat, curr_lon = user_102829.iloc[i]['lat'], user_102829.iloc[i]['lon']
    displacement = geodesic((prev_lat, prev_lon), (curr_lat, curr_lon)).km
    displacements.append(displacement)

max_displacement = max(displacements)
print(f"Maximum displacement for user 102829 on 24/05/2010 is {max_displacement:.2f} km.")

avg_displacement = sum(displacements) / len(displacements)
print(f"Average displacement for user 102829 on 24/05/2010 is {avg_displacement:.2f} km.")

total_displacement = sum(displacements)
print(f"Total displacement for user 102829 on 24/05/2010 is {total_displacement:.2f} km.")

In [ ]:
# Repeating the same steps used in the displacement lists above

dis2 = []

for i in range(len(user_75027) - 1):
    # obtaining nearest nodes in the network for the two consecutive points
    node1 = ox.nearest_nodes(graph, user_75027.iloc[i]['lon'], user_75027.iloc[i]['lat'])
    node2 = ox.nearest_nodes(graph, user_75027.iloc[i + 1]['lon'], user_75027.iloc[i + 1]['lat'])
    
    # calculating the shortest path length between the two nodes
    dis = nx.shortest_path_length(graph, node1, node2, weight='length')
    
    # appending the calculated maximum displacement to the list
    dis2.append(dis)

max_displacement = max(dis2)
print(f"Maximum displacement for user 75027 on 30/01/2010 is {max_displacement:.2f} m.")

avg_displacement = sum(dis2) / len(dis2)
print(f"Average displacement for user 75027 on 30/01/2010 is {avg_displacement:.2f} m.")

total_displacement = sum(dis2)
print(f"Total displacement for user 75027 on 30/01/2010 is {total_displacement:.2f} m.")

In [ ]:
# Repeating the same steps for user 102829

dis2 = []

for i in range(len(user_102829) - 1):
    node1 = ox.nearest_nodes(graph, user_102829.iloc[i]['lon'], user_102829.iloc[i]['lat'])
    node2 = ox.nearest_nodes(graph, user_102829.iloc[i + 1]['lon'], user_102829.iloc[i + 1]['lat'])
    
    dis = nx.shortest_path_length(graph, node1, node2, weight='length')
    
    dis2.append(dis)

max_displacement = max(dis2)
print(f"Maximum displacement for user 102829 on 24/05/2010 is {max_displacement:.2f} m.")

avg_displacement = sum(dis2) / len(dis2)
print(f"Average displacement for user 102829 on 24/05/2010 is {avg_displacement:.2f} m.")

total_displacement = sum(dis2)
print(f"Total displacement for user 102829 on 24/05/2010 is {total_displacement:.2f} m.")

In [ ]:
# A multi-graph is obtained using osmnx
G=ox.graph_from_place('Cambridge, United Kingdom', network_type='drive')

#It is then converted to a digraph
DG = ox.get_digraph(G)

#Its node centrality is then calculated
node_dc = nx.degree_centrality(DG)

# The node attributes are then set back to the edge
nx.set_node_attributes(DG, node_dc,'dc')

# Following which the digraph is converted to a multi-graph
G1 = nx.MultiGraph(DG)

# The closesness centrality is then calculated using the graph
edge_cc = nx.closeness_centrality(nx.line_graph(DG))

# And these attributes are set as the edge
nx.set_edge_attributes(DG,edge_cc,'cc')
G1 = nx.MultiGraph(DG)

# This graph is then plotted

nc = ox.plot.get_edge_colors_by_attr(G1, 'cc', cmap='plasma')
fig, ax = ox.plot_graph(G1, node_size=0, node_color='w', node_edgecolor='gray', node_zorder=2,
                        edge_color=nc, edge_linewidth=1.5, edge_alpha=1)

In [ ]:
# The graph is then converted to a geodatframe and is assigned a CRS which is rqeuired by contextily to plot a map underneath it
gdf_edges = ox.graph_to_gdfs(G1,nodes=False,fill_edge_geometry=True)
gdf_edges = gdf_edges.to_crs(epsg=3857) # setting crs to 3857

# Edges are then plotted accoridng to degree centrality
ax=gdf_edges.plot('cc',cmap='plasma',figsize=(10,10))

# A basemap is then added
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
plt.axis('off')
plt.show()

In [ ]:
# The geometries of the area, in this case the buildings, are obtained using the geometries_from_place function of the osmnx library
tags= tags={'amenity': True, 'building':True}
all_geom=ox.geometries.geometries_from_place('Cambridge, United Kingdom', tags)
all_geom = all_geom.to_crs(epsg=3857)

In [ ]:
# The first map here plots all buildings in the vicinity, which will be important while examining where to place a fire station later

fig,ax = plt.subplots(figsize=(10,10))
all_geom[all_geom['building'].notna()].plot(ax=ax,color='black')

gdf_edges = ox.graph_to_gdfs(G1, nodes=False, fill_edge_geometry=True)
gdf_edges = gdf_edges.to_crs(epsg=3857)

ax = gdf_edges.plot('cc', cmap='plasma', figsize=(10, 10), ax=ax)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
plt.axis('off')
plt.show()

The next two maps will represent the buildings in the area and the amenities in the area, from where information regarding residential buildings and the local fire station will be obtained.

In [ ]:
# Plotting the buildings and the closness centrality graph on top of each other

fig,ax = plt.subplots(figsize=(10,10))
all_geom[all_geom['building'].notna()].plot('building',
                                            ax=ax,
                                            categorical=True,
                                            legend=True)

gdf_edges = ox.graph_to_gdfs(G1, nodes=False, fill_edge_geometry=True)
gdf_edges = gdf_edges.to_crs(epsg=3857)

ax = gdf_edges.plot('cc', cmap='plasma', figsize=(10, 10), ax=ax)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
plt.axis('off')
plt.show()

In [ ]:
#This code isolates the fire station from the res of the amenities to make it easier to plot
fire_station = all_geom[(all_geom['amenity'] == 'fire_station')]

In [ ]:
# boolean indexing to isolate university building
# create list of building types to select
diff_types = ['apartments', 'dormitory', 'house', 'residential', 'student_residences', 'cabin', 'hotel']
fire_station[fire_station['amenity'].notna()].plot('amenity', ax=ax, categorical=True, legend=True)

# boolean indexing to select multiple building types
housing = all_geom[all_geom['building'].isin(diff_types)]

In [ ]:
# A graph of the selected residential buildings plotted without the centrality graph for better visibility

fig,ax = plt.subplots(figsize=(10,10))
housing[housing['building'].notna()].plot('building',
                                            ax=ax,
                                            categorical=True,
                                            legend=True)




import contextily as ctx
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)
plt.axis('off')
plt.show()

In [ ]:
# The same graph with centrality

fig, ax = plt.subplots(figsize=(10, 10))
housing[housing['building'].notna()].plot('building',
                                          ax=ax,
                                          categorical=True,
                                          legend=True)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

gdf_edges = ox.graph_to_gdfs(G1, nodes=False, fill_edge_geometry=True)
gdf_edges = gdf_edges.to_crs(epsg=3857)

ax = gdf_edges.plot('cc', cmap='plasma', figsize=(10, 10), ax=ax)

plt.axis('off')
plt.show()


In [ ]:
# This is the code to isolate the fire station from earlier, along with a code to plot fire stations in the area on contextily graphs

fire_station = all_geom[(all_geom['amenity'] == 'fire_station')]

fig, ax = plt.subplots(figsize=(10, 10))
fire_station[fire_station['amenity'].notna()].plot('amenity',
                                                   ax=ax,
                                                   categorical=True,
                                                   legend=True)

import contextily as ctx
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom=13)

plt.axis('off')
plt.show()


As there was only one fire station, this shows the need for adding at least another in order to provided better emergency services in the area

In [ ]:
# select the desired geometries
diff_types = ['apartments', 'dormitory', 'house', 'residential', 'student_residences', 'cabin', 'hotel']
buildings = all_geom[all_geom['building'].isin(diff_types)]
fire_stations = all_geom[all_geom['amenity'] == 'fire_station']

# plot the geometries and add a basemap
fig, ax = plt.subplots(figsize=(10, 10))
buildings.plot(column='building', categorical=True, legend=True, ax=ax)
fire_stations.plot(ax=ax, color='red', markersize=10)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)


gdf_edges = ox.graph_to_gdfs(G1, nodes=False, fill_edge_geometry=True)
gdf_edges = gdf_edges.to_crs(epsg=3857)

ax = gdf_edges.plot('cc', cmap='plasma', figsize=(10, 10), ax=ax)

plt.axis('off')
plt.show()


In a previous map that was produced, which has not been included in the code or report, the address of Cambridge was obtained using the `graph_from_address` function. That resulted in a map which was a fraction of the new map generated and the fire station was barely visible. In the new graph it is not visible at all.

To rectify this problem, the geometries are plotted on a folium map using the `geometries_from_address` function as using the place in general or using a distance of higher than 2500m would result in the kernel crashing

In [ ]:
# Geometries obtained similarly to earlier
small_geom=ox.geometries.geometries_from_address('Cambridge, United Kingdom', tags, dist = 2500)
small_geom = small_geom.to_crs(epsg=3857)

In [ ]:
# The residence buildings and fire station geometries are then added
diff_types = ['apartments', 'dormitory', 'house', 'residential', 'student_residences', 'cabin', 'hotel']
buildings = small_geom[small_geom['building'].isin(diff_types)]
fire_stations = small_geom[small_geom['amenity'] == 'fire_station']

# A folium map is created, similar to the one made earlier with geometries added as a GeoJson layer
map_cambridge = folium.Map(location=cambridge_coords, zoom_start=12)

folium.GeoJson(buildings).add_to(map_cambridge)
firestation_style = {'fillColor': 'red', 'color': 'red', 'fillOpacity': 0.7}
folium.GeoJson(fire_stations, style_function=lambda x: firestation_style).add_to(map_cambridge)

map_cambridge


The ideal place to build a new fire station can be obtained by using NearestNeighbours form the `Sci-kit Learn` library to calculate a point which would suffice and mark it on the folium map

In [ ]:
from shapely.geometry import Point

# Obtained the cambridge_buildings.geojson file online from OSM
json_geom = gpd.read_file("cambridge_buildings.geojson")

# Used the same steps as the previous map but with the json_geom file
diff_types = ['apartments', 'dormitory', 'house', 'residential', 'student_residences', 'cabin', 'hotel']
buildings = json_geom[json_geom['building'].isin(diff_types)]
fire_stations = json_geom[json_geom['amenity'] == 'fire_station']

map_cambridge = folium.Map(location=cambridge_coords, zoom_start=14)

# Importing the NearestNeighbors function from sklearn and numpy
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Creating an array from the geometry of the json files to fit in the function
X = np.array(list(zip(json_geom.geometry.x, json_geom.geometry.y)))
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

# Calculating the average distance to the nearest fire station for each building
json_geom['avg_distance'] = np.mean(distances[:,1:], axis=1)

# Finding the building with the greatest average distance to the fire station
best_location = json_geom.loc[json_geom['avg_distance'].idxmax()]

# Adding a marker for the ideal location as calculated using NearestNieghbors on the map
folium.Marker(
    location=[best_location.geometry.y, best_location.geometry.x],
    popup="Best location for new fire station",
    icon=folium.Icon(color='green'),
).add_to(map_cambridge)

# display the map
map_cambridge
